In [1]:
import os
import sys
# print(sys.path)
module_path = os.path.abspath(os.path.join('.'))
# print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
# print(sys.path)

## Initiate RAY

In [2]:
from ray.rllib.algorithms.a3c import A3CConfig
from ray import tune, air
import ray
from env.environment import MalariaEnvironment

In [3]:
ray.init(ignore_reinit_error=True, num_cpus=10,local_mode=True)
# ray.init()

2023-09-12 18:32:19,650	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.8.8
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8266


## Prepare the Environment

In [4]:
from ray.tune.registry import register_env

def env_creator(env_config):
    return MalariaEnvironment()  # return an env instance

register_env("MalariaEnvironment", env_creator)
env = MalariaEnvironment()

config = A3CConfig()

# config = config.training(lr=0.01) #.rollouts(num_rollout_workers=1)
# config = config.framework(framework='tf2')

# Set the config object's env.
config = config.environment(env="MalariaEnvironment")

In [5]:
from scipy.stats import linregress, sem

In [6]:
from ray.tune.logger import pretty_print

config = A3CConfig()
# Print out some default values.
# print(config.sample_async)  
# Update the config object.
config = config.training( 
    lr=tune.grid_search([0.01, 0.001, 0.0001]), 
    entropy_coeff=tune.grid_search([0.01, 0.001, 0.0001])
)
# config = config.training(lr=0.01)
# config = config.training(lr=tune.grid_search([0.001, 0.01, 0.1]))
# config = config.rollouts(num_rollout_workers=1)
config = config.framework()

# Set the config object's env.
config = config.environment(env="MalariaEnvironment") 
# Use to_dict() to get the old-style python config dict
# when running with tune.

#pretty_print(config.to_dict())

# algo = config.build()

In [ ]:
# https://docs.ray.io/en/latest/tune/examples/pbt_ppo_example.html
import random

import ray
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining

hyperparam_mutations = {
    "lambda": lambda: random.uniform(0.9, 1.0),
    "clip_param": lambda: random.uniform(0.01, 0.5),
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
#         "num_sgd_iter": lambda: random.randint(1, 30),
#         "sgd_minibatch_size": lambda: random.randint(128, 16384),
#         "train_batch_size": lambda: random.randint(2000, 160000),
}

pbt = PopulationBasedTraining(
    time_attr="time_total_s",
    perturbation_interval=120,
    resample_probability=0.25,
    # Specifies the mutations of these hyperparams
    hyperparam_mutations=hyperparam_mutations,
    #custom_explore_fn=explore,
)

# Stop when we've either reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100, "episode_reward_mean": -20}

tuner = tune.Tuner(
    "A3C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt
    ),
    param_space={
        "env": "MalariaEnvironment",
        "kl_coeff": 1.0,
        "num_workers": 4,
        "num_cpus": 1,  # number of CPUs to use per trial
        "num_gpus": 0,  # number of GPUs to use per trial
        "model": {"free_log_std": True},
        # These params are tuned from a fixed starting value.
        "lambda": tune.choice([0.95, 0.5, 0.2, 1]),#0.95,
        "clip_param": 0.2,
        "lr": tune.choice([1e-3, 5e-4, 1e-4, 5e-5, 1e-5]),#1e-4,
        # These params start off randomly drawn from a set.
        "num_sgd_iter": tune.choice([10, 20, 30]),
        "sgd_minibatch_size": tune.choice([128, 512, 2048]),
        "train_batch_size": tune.choice([10000, 20000, 40000]),
    },
    #run_config=air.RunConfig(stop=stopping_criteria),
)
results = tuner.fit()

<IPython.core.display.HTML object>
:task_name:bundle_reservation_check_func
:actor_name:A3C


:task_name:bundle_reservation_check_func
:actor_name:A3C


:actor_name:RolloutWorker
ACTION OrderedDict([('IRS', array([0.8715526], dtype=float32)), ('ITNS', array([0.65712345], dtype=float32))])
Setting IRS to 0.871552586555481
Setting ITNS to 0.6571234464645386


*** WRITING TO CAMPAIGNS FILE ***
*** SUCCESSFULLY CHANGED CAMPAIGNS FILE*** 


Action is setTrue

Starting Simulation...

/root/ray_results/A3C/A3C_MalariaEnvironment_a310a_00000_0_lambda=0.2000,lr=0.0001,num_sgd_iter=10,sgd_minibatch_size=128,train_batch_size=40000_2023-09-13_06-14-23


:actor_name:RolloutWorker


Intellectual Ventures(R)/EMOD Disease Transmission Kernel 2.20.17.0
Built on Apr  9 2019 by idm_bamboo_user from master(2d8a9f2) checked in on 2019-04-05 15:49:43 -0700
Supports sim_types: GENERIC, VECTOR, MALARIA, AIRBORNE, TBHIV, STI, HIV, PY, TYPHOID, ENVIRONMENTAL.

Using config file: config.json
Using input path: .
Using output path: output
Using dll path: 
Python not initialized because --python-script-path (-P) not set.
Initializing environment...
Log-levels:
	Default -> INFO
	Eradication -> INFO
00:00:00 [0] [I] [Eradication] Loaded Configuration...
00:00:00 [0] [I] [Eradication] 199 parameters found.
00:00:00 [0] [I] [Eradication] Initializing Controller...
00:00:00 [0] [I] [Controller] DefaultController::execute_internal()...
00:00:00 [0] [I] [RandomNumberGeneratorFactory] Using USE_PSEUDO_DES random number generator.
00:00:00 [0] [I] [Controller] DefaultController::execute_internal() populate simulation...
00:00:00 [0] [I] [Simulation] Campaign file name identified as: campa

...

Intellectual Ventures(R)/EMOD Disease Transmission Kernel 2.20.17.0
Built on Apr  9 2019 by idm_bamboo_user from master(2d8a9f2) checked in on 2019-04-05 15:49:43 -0700
Supports sim_types: GENERIC, VECTOR, MALARIA, AIRBORNE, TBHIV, STI, HIV, PY, TYPHOID, ENVIRONMENTAL.

Using config file: config.json
Using input path: .
Using output path: output
Using dll path: 
Python not initialized because --python-script-path (-P) not set.
Initializing environment...
Log-levels:
	Default -> INFO
	Eradication -> INFO
00:00:00 [0] [I] [Eradication] Loaded Configuration...
00:00:00 [0] [I] [Eradication] 199 parameters found.
00:00:00 [0] [I] [Eradication] Initializing Controller...
00:00:00 [0] [I] [Controller] DefaultController::execute_internal()...
00:00:00 [0] [I] [RandomNumberGeneratorFactory] Using USE_PSEUDO_DES random number generator.
00:00:00 [0] [I] [Controller] DefaultController::execute_internal() populate simulation...
00:00:00 [0] [I] [Simulation] Campaign file name identified as: campa

Intellectual Ventures(R)/EMOD Disease Transmission Kernel 2.20.17.0
Built on Apr  9 2019 by idm_bamboo_user from master(2d8a9f2) checked in on 2019-04-05 15:49:43 -0700
Supports sim_types: GENERIC, VECTOR, MALARIA, AIRBORNE, TBHIV, STI, HIV, PY, TYPHOID, ENVIRONMENTAL.

Using config file: config.json
Using input path: .
Using output path: output
Using dll path: 
Python not initialized because --python-script-path (-P) not set.
Initializing environment...
Log-levels:
	Default -> INFO
	Eradication -> INFO
00:00:00 [0] [I] [Eradication] Loaded Configuration...
00:00:00 [0] [I] [Eradication] 199 parameters found.
00:00:00 [0] [I] [Eradication] Initializing Controller...
00:00:00 [0] [I] [Controller] DefaultController::execute_internal()...
00:00:00 [0] [I] [RandomNumberGeneratorFactory] Using USE_PSEUDO_DES random number generator.
00:00:00 [0] [I] [Controller] DefaultController::execute_internal() populate simulation...
00:00:00 [0] [I] [Simulation] Campaign file name identified as: campa

Intellectual Ventures(R)/EMOD Disease Transmission Kernel 2.20.17.0
Built on Apr  9 2019 by idm_bamboo_user from master(2d8a9f2) checked in on 2019-04-05 15:49:43 -0700
Supports sim_types: GENERIC, VECTOR, MALARIA, AIRBORNE, TBHIV, STI, HIV, PY, TYPHOID, ENVIRONMENTAL.

Using config file: config.json
Using input path: .
Using output path: output
Using dll path: 
Python not initialized because --python-script-path (-P) not set.
Initializing environment...
Log-levels:
	Default -> INFO
	Eradication -> INFO
00:00:00 [0] [I] [Eradication] Loaded Configuration...
00:00:00 [0] [I] [Eradication] 199 parameters found.
00:00:00 [0] [I] [Eradication] Initializing Controller...
00:00:00 [0] [I] [Controller] DefaultController::execute_internal()...
00:00:00 [0] [I] [RandomNumberGeneratorFactory] Using USE_PSEUDO_DES random number generator.
00:00:00 [0] [I] [Controller] DefaultController::execute_internal() populate simulation...
00:00:00 [0] [I] [Simulation] Campaign file name identified as: campa

/root/ray_results/A3C/A3C_MalariaEnvironment_a310a_00000_0_lambda=0.2000,lr=0.0001,num_sgd_iter=10,sgd_minibatch_size=128,train_batch_size=40000_2023-09-13_06-14-23
Intellectual Ventures(R)/EMOD Disease Transmission Kernel 2.20.17.0
Built on Apr  9 2019 by idm_bamboo_user from master(2d8a9f2) checked in on 2019-04-05 15:49:43 -0700
Supports sim_types: GENERIC, VECTOR, MALARIA, AIRBORNE, TBHIV, STI, HIV, PY, TYPHOID, ENVIRONMENTAL.

Using config file: config.json
Using input path: .
Using output path: output
Using dll path: 
Python not initialized because --python-script-path (-P) not set.
Initializing environment...
Log-levels:
	Default -> INFO
	Eradication -> INFO
00:00:00 [0] [I] [Eradication] Loaded Configuration...
00:00:00 [0] [I] [Eradication] 199 parameters found.
00:00:00 [0] [I] [Eradication] Initializing Controller...
00:00:00 [0] [I] [Controller] DefaultController::execute_internal()...
00:00:00 [0] [I] [RandomNumberGeneratorFactory] Using USE_PSEUDO_DES random number genera

Intellectual Ventures(R)/EMOD Disease Transmission Kernel 2.20.17.0
Built on Apr  9 2019 by idm_bamboo_user from master(2d8a9f2) checked in on 2019-04-05 15:49:43 -0700
Supports sim_types: GENERIC, VECTOR, MALARIA, AIRBORNE, TBHIV, STI, HIV, PY, TYPHOID, ENVIRONMENTAL.

Using config file: config.json
Using input path: .
Using output path: output
Using dll path: 
Python not initialized because --python-script-path (-P) not set.
Initializing environment...
Log-levels:
	Default -> INFO
	Eradication -> INFO
00:00:00 [0] [I] [Eradication] Loaded Configuration...
00:00:00 [0] [I] [Eradication] 199 parameters found.
00:00:00 [0] [I] [Eradication] Initializing Controller...
00:00:00 [0] [I] [Controller] DefaultController::execute_internal()...
00:00:00 [0] [I] [RandomNumberGeneratorFactory] Using USE_PSEUDO_DES random number generator.
00:00:00 [0] [I] [Controller] DefaultController::execute_internal() populate simulation...
00:00:00 [0] [I] [Simulation] Campaign file name identified as: campa

In [ ]:
import pprint

best_result = results.get_best_result()

print("Best performing trial's final set of hyperparameters:\n")
pprint.pprint(
    {k: v for k, v in best_result.config.items() if k in hyperparam_mutations}
)

print("\nBest performing trial's final reported metrics:\n")

metrics_to_print = [
    "episode_reward_mean",
    "episode_reward_max",
    "episode_reward_min",
    "episode_len_mean",
]
pprint.pprint({k: v for k, v in best_result.metrics.items() if k in metrics_to_print})

In [ ]:
df = results.get_dataframe()

In [ ]:
df.head()